<a href="https://colab.research.google.com/github/premviaemail/Finance/blob/main/LoanFeeAmortization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install numpy_financial

In [10]:
import numpy as np
import numpy_financial as npf
import pandas as pd

# Constants
loan_principal = 100000
origination_fee = 3000
direct_loan_costs = 1000
Payments = 16275
stated_interest_rate = 0.10
loan_term = 10

# Format: [remaining principal, year]
actual_principal_balances = [
  [93725, 1] , # End of Year 1: Principal 93725
  [93725, 2],  # End of Year 2: Principal 93725
  [93725, 3]  # End of Year 3: Principal 93725
]
# Initial values
initial_carrying_amount = loan_principal - origination_fee + direct_loan_costs
remaining_principal = loan_principal
unamortized_net_fees = origination_fee - direct_loan_costs
carrying_amount = initial_carrying_amount
total_amortization = 0
prepayment_amount = 0  # The amount of the prepayment

# Calculate the periodic payment amount
periodic_payment = -npf.pmt(stated_interest_rate, loan_term, loan_principal, 0, 'end')

# Initialize cash flows with the initial carrying value
cash_flows = [-initial_carrying_amount]

# Update cash flows with periodic payments
periodic_cash_flows = [periodic_payment] * loan_term
cash_flows.extend(periodic_cash_flows)

# Update effective_interest_rate periodic payments
effective_interest_rate = npf.irr(cash_flows)
print("Effective Interest Rate:", effective_interest_rate)

# Lists to store the schedule
periods = [0]  # Start with period 0
cash_inflow = [-initial_carrying_amount]  # No cash inflow in period 0
stated_interest = [0]  # No stated interest in period 0
amortization = [0]  # No amortization in period 0
interest_income = [0]  # No interest income in period 0
remaining_principal_list = [loan_principal]  # Initial principal in period 0
prepayment_amount_list = [0]  # Initial prepayment amount in period 0
unamortized_net_fees_list = [unamortized_net_fees]  # Initial unamortized net fees in period 0
carrying_amount_list = [initial_carrying_amount]  # Initial carrying amount in period 0
payment_period_list = [loan_term]  # Initialize with original loan term in months

year = 1
while year <= loan_term or remaining_principal > 0:
    # Calculate stated interest
    annual_payment = Payments
    if year == 1:
        prior_remaining_principal = loan_principal
        prior_carrying_amount = initial_carrying_amount
        prior_unamortized_net_fees = unamortized_net_fees
        prior_payment_periods = loan_term
    else:
        prior_remaining_principal = remaining_principal_list[-1]
        prior_carrying_amount = carrying_amount_list[-1]
        prior_unamortized_net_fees = unamortized_net_fees_list[-1]
        prior_payment_periods = payment_period_list[-1]

    # Calculate stated interest & expected_remaining_principal
    stated_int = prior_remaining_principal * stated_interest_rate
    expected_remaining_principal = prior_remaining_principal - (annual_payment - stated_int)

    # Update remaining principal with actual values for the first periods
    if year <= len(actual_principal_balances):
        remaining_principal, actual_year = actual_principal_balances[year-1]
        if remaining_principal == prior_remaining_principal:
            prepayment_amount = 0
            annual_payment = 0
            stated_int = 0
        else:
            prepayment_amount = expected_remaining_principal - remaining_principal
    else:
        remaining_principal = expected_remaining_principal
        prepayment_amount = 0


    # Calculate payment periods
    payment_periods = npf.nper(stated_interest_rate, -periodic_payment, remaining_principal, 0, 'end')
    carrying_amount = -npf.pv(effective_interest_rate, payment_periods, periodic_payment, 0, 'end')


    # Handle payment_periods = 0
    if payment_periods <= 0:
        carrying_amount = 0
        remaining_principal = 0
        payment_periods=0

     # Calculate amortization
    if annual_payment == 0:  # Missed payment
        unamortized_net_fees = prior_unamortized_net_fees
        amort = 0  # No amortization when no payment is made
    elif payment_periods <= 0:
        amort = prior_unamortized_net_fees
    else:
        unamortized_net_fees = remaining_principal - carrying_amount
        amort = prior_unamortized_net_fees - unamortized_net_fees


    # Calculate interest income
    int_income = stated_int + amort

    # Handle prior_payment_periods = 0
    if prior_payment_periods <= 0:
        remaining_principal = 0
        carrying_amount = 0
        unamortized_net_fees = 0
        stated_int = 0
        int_income = 0
        amort = 0
        payment_periods = 0
        annual_payment = 0


 # Filter based on the conditions directly within the loop
    if annual_payment > 0:
        periods.append(year)
        cash_inflow.append(round(annual_payment + prepayment_amount))
        prepayment_amount_list.append(round(prepayment_amount))
        stated_interest.append(round(stated_int))
        amortization.append(round(amort))
        interest_income.append(round(int_income))
        remaining_principal_list.append(round(remaining_principal))
        unamortized_net_fees_list.append(round(unamortized_net_fees))
        carrying_amount_list.append(round(carrying_amount))
        payment_period_list.append(np.round(payment_periods, 3))


   # Increment the year
    year += 1

# Create a DataFrame
schedule = pd.DataFrame({
    "Period": periods,
    "Cash Inflow (Outflow)": cash_inflow,
    "Prepayment Amount": prepayment_amount_list,
    "Stated Interest": stated_interest,
    "Amortization": amortization,
    "Interest Income": interest_income,
    "Remaining Principal": remaining_principal_list,
    "Unamortized Net Fees": unamortized_net_fees_list,
    "Carrying Amount": carrying_amount_list,
    "Payment Periods": payment_period_list
})

# Print the schedule
print(schedule.to_csv(sep='\t', index=False))

# Total amortization
total_amortization = sum(amortization)
print("\nTotal Amortization:", round(total_amortization, 2))


Effective Interest Rate: 0.10472982887457327
Period	Cash Inflow (Outflow)	Prepayment Amount	Stated Interest	Amortization	Interest Income	Remaining Principal	Unamortized Net Fees	Carrying Amount	Payment Periods
0	-98000	0	0	0	0	100000	2000	98000	10.0
1	16275	0	10000	264	10264	93725	1736	91989	9.0
4	16275	0	9372	261	9634	86822	1475	85348	8.0
5	16275	0	8682	256	8938	79229	1219	78010	7.0
6	16275	0	7923	247	8170	70877	972	69905	6.0
7	16275	0	7088	234	7322	61690	738	60952	5.0
8	16275	0	6169	214	6383	51584	524	51060	4.0
9	16275	0	5158	189	5348	40467	335	40133	3.0
10	16275	0	4047	157	4203	28239	178	28060	2.0
11	16275	0	2824	115	2939	14788	63	14725	0.999
12	16275	0	1479	63	1542	0	63	0	0.0


Total Amortization: 2000
